### Ingestion del archivo "country.json"

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

#### Paso 1 - Leer el archivo JSON utilizando "DataFrameReader" de Spark

In [0]:
countries_schema = "countryId INT, countryIsoCode STRING, countryName STRING"

In [0]:
countries_df = spark.read \
    .schema(countries_schema)\
    .json(f"{bronze_folder_path}/{v_file_date}/country.json")

#### Paso 2 - Eliminar columnas no deseadas del dataframe

In [0]:
countries_dropped_df = countries_df.drop("countryIsoCode")

#### Paso 3 - Cambiar el nombre de las columnas segun lo "requerido"

In [0]:
countries_renamed_df = countries_dropped_df \
    .withColumnRenamed("countryId","country_id") \
    .withColumnRenamed("countryName","country_name")

#### Paso 3 - Agregar una columna "ingestion date" al dataframe

In [0]:
from pyspark.sql.functions import current_timestamp
from pyspark.sql.functions import lit

In [0]:
country_final_df = add_ingestion_date(countries_renamed_df) \
.withColumn("enviroment", lit(dbutils.widgets.get("p_enviroment")))\
.withColumn("file_date", lit(v_file_date))


#### Paso 4 - Escribir datos en Datalake en formato parquet

In [0]:
#country_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/country")
country_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.country");

In [0]:
%sql
SELECT * FROM movie_silver.country